# Part 0: Dataloader and Visualizations

In [3]:
!pwd # currently in /content folder
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar

/content
--2021-03-12 05:36:28--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar.1’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  13.5MB/s    in 33s     

2021-03-12 05:37:01 (13.1 MB/s) - ‘VOCtrainval_06-Nov-2007.tar.1’ saved [460032000/460032000]



In [4]:
!tar xf VOCtrainval_06-Nov-2007.tar

In [5]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar && tar xf VOCtest_06-Nov-2007.tar

--2021-03-12 05:37:35--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 451020800 (430M) [application/x-tar]
Saving to: ‘VOCtest_06-Nov-2007.tar’

VOCtest_06-Nov-2007 100%[===================>] 430.13M  13.3MB/s    in 33s     

2021-03-12 05:38:08 (13.1 MB/s) - ‘VOCtest_06-Nov-2007.tar’ saved [451020800/451020800]



In [6]:
!cd VOCdevkit/VOC2007
!export DATA_DIR=$(pwd)

In [9]:
!pwd # back on /content
!cd VOCdevkit/VOC2007/
# !wget http://www.cs.cmu.edu/~spurushw/hw2_files/selective_search_data.tar
# !tar xf selective_search_data.tar

/content
--2021-03-12 05:40:43--  http://www.cs.cmu.edu/~spurushw/hw2_files/selective_search_data.tar
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-03-12 05:40:44 ERROR 404: Not Found.

tar: selective_search_data.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import torch
import wandb
import scipy.io

import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

from voc_dataset import VOCDataset

from PIL import Image

from utils import *

USE_WANDB = False

## Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself. Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf).

Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals. You should be able to access the `.mat` files using `scipy.io.loadmat('file.mat')`. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc.

Your task is to change the dataloader to obtain the ground-truth bounding boxes, as well as the proposed bounding boxes for each image. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [ ]:
# Load the Dataset - items at a particular index can be accesed by usual indexing notation (dataset[idx])
dataset = VOCDataset('trainval', top_n=10)

In [ ]:
#TODO: get the image information from index 2020
idx = 2020

# input = 

## Wandb Init and Logging
Initialize a Weights and Biases project, and convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions).

You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [ ]:
if USE_WANDB:
    wandb.init(project="vlr2", reinit=True)

See this block as an example of plotting the ground truth box for an image.

In [ ]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))

img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})

Check the `get_box_data` function in `utils.py` and understand how it is being used. Log the image with the GT bounding box on wandb.
After, this you should be able to easily plot the top 10 bounding proposals as well.

In [ ]:
nums = range(len(rois)) # placeholder for names of proposals

#TODO: plot top ten proposals (of bounding boxes)